In [1]:
# import importlib

import numpy as np
import os

import plugins.custom_config as custom_config
import plugins.bot_core as bot_core
import plugins.tool_dice as tool_dice
import plugins.help_info as help_info
import plugins.utils as utils

from plugins import bot

b = bot

# importlib.reload(bot_tool)
# importlib.reload(dice_tool)
# importlib.reload(utils)

ujson module not found, using json


个人资料库加载成功!
找到以下查询资料: ['5e_00_spell_feat_phb_exp.json', '5e_01_armor_weapon_phb_by_xe.json', '5e_01_phb_classes_by_pear_hze_xe.json', '5e_01_phb_races_by_pear.json', '5e_02_phb_rule_by_pear.json', '5e_03_tools_phb_by_zxa.json', '5e_05_phb_spell_list_by_pear.json', '5e_06_magic_items_dmg_by_hze.json', '5e_07_dmg_rule_by_pear.json', '5e_08_mm_by_hze_pear_query.json', '5e_08_phb_background_by_pear.json', '5e_09_creature_phb_dmg_by_hze.json', '5e_10_exp_classes_by_hh_pear.json', '5e_10_exp_magic_items_by_hh.json', '5e_10_exp_races_by_pear.json', '5e_10_xge_dm_tools_by_pear.json', '5e_19_egtw_by_pear_3_29.json', '5e_30_module_items_by_hh.json', 'syn_spell.json']
成功加载5e_00_spell_feat_phb_exp.json, 共546个条目
成功加载5e_01_armor_weapon_phb_by_xe.json, 共69个条目
成功加载5e_01_phb_classes_by_pear_hze_xe.json, 共582个条目
成功加载5e_01_phb_races_by_pear.json, 共115个条目
成功加载5e_02_phb_rule_by_pear.json, 共203个条目
成功加载5e_03_tools_phb_by_zxa.json, 共115个条目
成功加载5e_05_phb_spell_list_by_pear.json, 共70个条目
成功加载5e_06_magic_items_dmg

In [2]:
assert utils.ChineseToEnglishSymbol('。') == '.'
assert utils.ChineseToEnglishSymbol('.') == '.'
assert utils.int2str(4) == '+4'
assert utils.int2str(0) == '+0'
assert utils.int2str(-1) == '-1'

assert await bot_core.ParseInput('.rd20')
assert await bot_core.ParseInput(' . r D20 reason')
assert await bot_core.ParseInput('.r')
assert await bot_core.ParseInput('.rh')
assert await bot_core.ParseInput('.rs')
assert await bot_core.ParseInput('abc') is None
assert await bot_core.ParseInput('') is None
assert await bot_core.ParseInput('测试') is None

In [3]:
assert tool_dice.RollDiceCommand('12') [1] == '12=12'
assert tool_dice.RollDiceCommand('4d6+2')[0] == 0 and tool_dice.RollDiceCommand('4d6+2') [1][:6] == '4D6+2='
assert tool_dice.RollDiceCommand('d20') [0] == 0 and tool_dice.RollDiceCommand('d20') [1][:3] == 'D20'
assert tool_dice.RollDiceCommand('d') [0] == 0 and tool_dice.RollDiceCommand('d') [1][:3] == 'D20'
assert tool_dice.RollDiceCommand('1D4') [0] == 0
assert tool_dice.RollDiceCommand('30D4') [0] == 0
assert tool_dice.RollDiceCommand('D20优势+2') [0] == 0 and tool_dice.RollDiceCommand('D20优势+2') [1][:12] == 'D20优势+2=Max('
assert tool_dice.RollDiceCommand('D20劣势-4') [0] == 0 and tool_dice.RollDiceCommand('D20劣势-4') [1][:12] == 'D20劣势-4=Min('
assert tool_dice.RollDiceCommand('4d6k3') [0] == 0 and tool_dice.RollDiceCommand('4d6k3') [1][:11] == '4D6K3=Max3('
assert tool_dice.RollDiceCommand('6#4d6k3') [0] == 0 and tool_dice.RollDiceCommand('6#4d6k3') [1][:14] == '6次4D6K3={\nMax3'
assert tool_dice.RollDiceCommand('2d20k1+5+1d4') [0] == 0 and tool_dice.RollDiceCommand('2d20k1+5+1d4') [1][:17] == '2D20K1+5+1D4=Max('
assert tool_dice.RollDiceCommand('d优势+2')[1][:12] == 'D20优势+2=Max('
assert tool_dice.RollDiceCommand('3dk1+5抗性')[1][:16] == '3D20K1+5抗性=(Max('

In [4]:
assert tool_dice.SplitDiceCommand('d20优势+5攻击地精') == ('d20优势+5', '攻击地精')
assert tool_dice.SplitDiceCommand('d20优势+5     攻击地精') == ('d20优势+5', '攻击地精')
assert tool_dice.SplitDiceCommand('d20优势+5     攻击地精') == ('d20优势+5', '攻击地精')
assert tool_dice.SplitDiceCommand('d20优势+5') == ('d20优势+5', '')

In [5]:
assert (await b.ProcessMessage('.nn', '821480843', 'ABCD'))[0].resultStr
assert (await b.ProcessMessage('.rd20', '821480843', 'TEST_USER'))[0].resultStr
assert (await b.ProcessMessage('.rd', '821480843', 'TEST_USER'))[0].resultStr
assert (await b.ProcessMessage('.rd     ', '821480843', 'TEST_USER'))[0].resultStr
assert (await b.ProcessMessage('.rd    未知原因', '821480843', 'TEST_USER'))[0].resultStr
assert (await b.ProcessMessage('.rd20攻击', '821480843', 'TEST_USER'))[0].resultStr
assert (await b.ProcessMessage('.rd优势', '821480843', 'TEST_USER'))[0].resultStr
assert (await b.ProcessMessage('.rd+5', '821480843', 'TEST_USER'))[0].resultStr
assert (await b.ProcessMessage('.rs6d20', '82148084', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.rs4#6d20', '82148084', 'TEST_USER', 'groupId_test'))[0].resultStr

In [6]:
assert (await b.ProcessMessage('.nn', '821480843', 'ABCD'))[0].resultStr == '要用本来的名字称呼你吗? 了解!'
assert (await b.ProcessMessage('.nn 西瓜', '821480843', 'ABCD'))[0].resultStr == '要称呼你为西瓜吗? 没问题!'
assert (await b.ProcessMessage('.rd20', '821480843', 'TEST_USER'))[0].resultStr[:9] == '西瓜掷出了D20='
assert (await b.ProcessMessage('.nn 冬瓜', '821480843', 'ABCD'))[0].resultStr == '要称呼你为冬瓜吗? 没问题!'
assert (await b.ProcessMessage('.rd20', '821480843', 'TEST_USER'))[0].resultStr[:9] == '冬瓜掷出了D20='
assert ((await b.ProcessMessage('.jrrp', '821480843', 'TEST_USER'))[0].resultStr)
assert (await b.ProcessMessage('.nn', '821480843', 'ABCD'))[0].resultStr == '要用本来的名字称呼你吗? 了解!'
assert (await b.ProcessMessage('.rd20', '821480843', 'TEST_USER'))[0].resultStr[:16] == 'TEST_USER掷出了D20='
assert ((await b.ProcessMessage('.jrrp', '821480843', 'TEST_USER'))[0].resultStr)

In [7]:
assert (await b.ProcessMessage('.nn', '821480843', 'ABCD'))[0].resultStr == '要用本来的名字称呼你吗? 了解!'
assert (await b.ProcessMessage('.nn 格鲁特', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr == '要称呼你为格鲁特吗? 没问题!'
assert (await b.ProcessMessage('.hp 40', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr == '成功将格鲁特的生命值设置为40=40'
assert (await b.ProcessMessage('.hp 40/50', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr == '成功将格鲁特的生命值设置为40=40, 最大生命值是50'
assert (await b.ProcessMessage('.init clr', '821480843', 'TEST_USER', 'groupId_test'))[0]
assert (await b.ProcessMessage('.init', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr == '还没有做好先攻列表哦'
assert (await b.ProcessMessage('.ri+2', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr[:12] == '格鲁特先攻:D20+2='
assert (await b.ProcessMessage('.ri-1', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr[:12] == '格鲁特先攻:D20-1='
assert (await b.ProcessMessage('.ri', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr[:10] == '格鲁特先攻:D20='
assert (await b.ProcessMessage('.ri-2地精', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr[:11] == '地精先攻:D20-2='
assert (await b.ProcessMessage('.ri+1 1d4+1#大僵尸/小僵尸/快速僵尸+2', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.ri劣势+1d4 地精', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr[:19] =='地精先攻:D20劣势+1D4=Min('
assert (await b.ProcessMessage('.ri20 巢穴', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr == '巢穴先攻:20=20'
assert (await b.ProcessMessage('.ri-5', '87654321', '米达尔', 'groupId_test'))[0].resultStr[:12] == '米达尔先攻:D20-5='
assert (await b.ProcessMessage('.init', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.init del 地精', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr == '已经将地精从先攻列表中删除'
assert (await b.ProcessMessage('.init del米达尔', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr == '已经将米达尔从先攻列表中删除'
assert (await b.ProcessMessage('.init', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.init clr', '821480843', 'TEST_USER', 'groupId_test'))[0]
assert (await b.ProcessMessage('.init', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr == '还没有做好先攻列表哦'
assert (await b.ProcessMessage('.nn', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr == '要用本来的名字称呼你吗? 了解!'
assert (await b.ProcessMessage('.rd20', '821480843', 'ABCD'))[0].resultStr[:11] == 'ABCD掷出了D20='
assert (await b.ProcessMessage('.hp clr', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr == '已经忘记了TEST_USER的生命值...'

In [8]:
assert (await b.ProcessMessage('.nn 格鲁特', '821480843', 'TEST_USER'))[0].resultStr == '要称呼你为格鲁特吗? 没问题!'
assert (await b.ProcessMessage('.ri', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.ri', '87654321', '米达尔', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.ri 地精', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.ri 卓尔精灵A', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.init', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.hp 20/40', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.hp 地精- 1d8+4', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.hp 格+ 1d8+3', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.hp10', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.hp A-1d6+10', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.hpA2d8', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.hpA-20', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.hp米-3d4+3', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert ((await b.ProcessMessage('.米-3d4+3hp', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)
assert ((await b.ProcessMessage('.地精hp- 1d8+4', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)
assert ((await b.ProcessMessage('.米/地精hp-4d8', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)
assert (await b.ProcessMessage('.init', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.init clr', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.hp clr', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.hp clr', '87654321', '米达尔', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.nn', '821480843', 'TEST_USER'))[0].resultStr == '要用本来的名字称呼你吗? 了解!'

In [9]:
assert (await b.ProcessMessage('.init clr', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.ri', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.hp 10', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.hp 20', '12345678', 'TEST_USER2', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.ri 约翰', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.nn 约瑟夫', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.nn 约翰', '12345678', 'TEST_USER2', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.init', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.nn', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.nn', '12345678', 'TEST_USER2', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.init clr', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr

In [10]:
assert (await b.ProcessMessage('.bot on', '821480843', 'TEST_USER', 'groupId_test')) == None
assert (await b.ProcessMessage('.bot on', '821480843', 'TEST_USER', 'groupId_test', True))[0].resultStr
assert (await b.ProcessMessage('.bot off', '821480843', 'TEST_USER', 'groupId_test', True))[0].resultStr
assert (await b.ProcessMessage('.rd20沉默', '821480843', 'TEST_USER', 'groupId_test')) == None
assert (await b.ProcessMessage('.rd20沉默', '821480843', 'TEST_USER', 'groupId_test', True))[0] != None
assert (await b.ProcessMessage('.bot on', '821480843', 'TEST_USER', 'groupId_test', True))[0].resultStr
assert (await b.ProcessMessage('.rd20沉默', '821480843', 'TEST_USER', 'groupId_test'))[0] != None

In [11]:
assert (await b.ProcessMessage('.dnd', '821480843', 'TEST_USER'))[0]
assert (await b.ProcessMessage('.dnd 3', '821480843', 'TEST_USER', 'groupId_test'))[0]
assert (await b.ProcessMessage('.dnd3 大力出奇迹', '821480843', 'TEST_USER', 'groupId_test'))[0]

In [12]:
assert (await b.ProcessMessage('.help', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr == help_info.HELP_STR
assert (await b.ProcessMessage('.help指令', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr == help_info.HELP_COMMAND_STR
assert (await b.ProcessMessage('.help  指 令', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr == help_info.HELP_COMMAND_STR
assert (await b.ProcessMessage('.help链接', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr == help_info.HELP_LINK_STR
assert (await b.ProcessMessage('.help协议', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr == help_info.HELP_AGREEMENT_STR
assert (await b.ProcessMessage('.help查询', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr == help_info.HELP_COMMAND_QUERY_STR
assert (await b.ProcessMessage('.help draw', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr == help_info.HELP_COMMAND_DRAW_STR

In [13]:
assert (await b.ProcessMessage('.查询火球术', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr[:11] == '火球术Fireball'
assert (await b.ProcessMessage('.查询  火球术', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr[:11] == '火球术Fireball'
assert (await b.ProcessMessage('.查询  火 球 术', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr[:11] == '火球术Fireball'
assert (await b.ProcessMessage('.查询火球', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr.find('找到多个匹配的条目:') != -1
assert (await b.ProcessMessage('.查询   ', '821480843', 'TEST_USER', 'groupId_test'))[0]
assert (await b.ProcessMessage('.查询 高/隐', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr[:13]

In [14]:
assert (await b.ProcessMessage('.rhd20优势', '821480843', 'TEST_USER', 'groupId_test'))[0].personIdList == ['821480843']
assert (await b.ProcessMessage('.rhd20优势', '821480843', 'TEST_USER', 'groupId_test'))[0].groupIdList is None
assert (await b.ProcessMessage('.r1d8抗性', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.r1d8+2d10抗性', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.r20易伤', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.rd+2d8+1d4易伤', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.r3#1d8+1d12抗性', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.r易伤d+2d8+1d4', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.r3dk1易伤', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr

In [15]:
assert (await b.ProcessMessage('.draw 狂野', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.draw狂野魔法浪涌表', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.draw', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert ((await b.ProcessMessage('.draw万象', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)
assert ((await b.ProcessMessage('.draw天气', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)
assert ((await b.ProcessMessage('.draw库藏宝藏cr0', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)
assert ((await b.ProcessMessage('.draw11#库藏宝藏cr0', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)

In [16]:
assert (await b.ProcessMessage('.烹饪', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.烹饪-4', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.烹饪优势+1d4', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.烹饪 经典/主菜', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.烹饪 野炊', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.烹饪 经典/主菜////精灵/黑暗/奢侈', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr

In [17]:
assert (await b.ProcessMessage('.点菜', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.点菜1d4', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.点菜3', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.点菜5经典', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.点菜 经典/主菜////精灵/黑暗/奢侈', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.今日菜单', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr

In [18]:
template = (await b.ProcessMessage('.角色卡模板', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage(f'.记录角色卡 {template}', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert ((await b.ProcessMessage('.法术位', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)
assert ((await b.ProcessMessage('.2环法术位-1', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)
assert ((await b.ProcessMessage('.1环-1', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)
assert ((await b.ProcessMessage('.金钱-52sp-20cp', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)
assert ((await b.ProcessMessage('.金钱-5', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)
assert (await b.ProcessMessage(f'.金钱', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert ((await b.ProcessMessage('.查看角色卡 ', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)
assert ((await b.ProcessMessage('.长休', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)
assert ((await b.ProcessMessage('.清除法术位', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)
assert ((await b.ProcessMessage('.清除金钱', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)
assert ((await b.ProcessMessage('.查看角色卡 ', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)
assert ((await b.ProcessMessage('.完整角色卡 ', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)
assert (await b.ProcessMessage('.敏捷检定+4+1d4 躲藏', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert ((await b.ProcessMessage('.3#敏捷检定+2', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)
assert ((await b.ProcessMessage('.3#敏捷攻击', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)
assert ((await b.ProcessMessage('.3#敏捷豁免', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)
assert (await b.ProcessMessage('.敏捷豁免+2', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.察觉检定', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert ((await b.ProcessMessage('.敏捷攻击', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)
assert ((await b.ProcessMessage('.长休', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)
assert (await b.ProcessMessage('.清除角色卡 ', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.nn', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.角色卡', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.说服检定', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.长休', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr

In [19]:
assert (await b.ProcessMessage('.credit 12345678:+10', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.credit 12345678:-10', '821480843', 'TEST_USER', 'groupId_test'))[1].resultStr
print ((await b.ProcessMessage('.好感度', '12345678', 'TEST_USER', 'groupId_test'))[0].resultStr)
print ((await b.ProcessMessage('.好感度', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)
print ((await b.ProcessMessage('.credit 821480843', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)
# assert (await b.ProcessMessage('.savedata', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert ((await b.ProcessMessage('.dailyprofile', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)
assert (await b.ProcessMessage('.今日笑话', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.今日笑话', '12345678', 'TEST_USER', 'groupId_test'))[0].resultStr

对TEST_USER的好感度为3~
对TEST_USER的好感度为637~
821480843的好感度是637


In [20]:
assert (await b.ProcessMessage('.记录笔记 包子', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.记录笔记 馒头', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.记录笔记 1:面条', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.笔记 xxyyzz', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.笔记 临时', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.笔记', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.清除笔记1', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.清除笔记 所有笔记', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.笔记', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr

In [21]:
assert (await b.ProcessMessage('.welcome 欢迎~', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert b.GetWelcome('groupId_test')

In [22]:
b.groupInfoDict

{'922755017': {'active': True,
  'credit': 0,
  'commandAccu': 0,
  'welcome': '欢迎新人~',
  'warning': 1,
  'noticeBool': True,
  'noticeStr': '更新到v0.5.9.2版本咯~ 输入.help更新 查看详细说明吧~',
  'note': {},
  'messageAccu': 0,
  'dndCommandAccu': 0,
  'commandDaily': 0,
  'messageDaily': 0,
  'dndCommandDaily': 0,
  'name': '未知群名称'},
 '861919492': {'active': True,
  'credit': 0,
  'commandAccu': 4,
  'welcome': '欢迎加入伊丽莎白粉丝群~',
  'warning': 0,
  'noticeBool': False,
  'noticeStr': '更新到v0.5.9.2版本咯~ 输入.help更新 查看详细说明吧~',
  'note': {},
  'messageAccu': 73,
  'dndCommandAccu': 1,
  'commandDaily': 4,
  'messageDaily': 73,
  'dndCommandDaily': 1,
  'name': '未知群名称'},
 '588983918': {'active': True,
  'credit': 0,
  'commandAccu': 0,
  'welcome': '欢迎新人~',
  'warning': 0,
  'noticeBool': True,
  'noticeStr': '更新到v0.5.9.2版本咯~ 输入.help更新 查看详细说明吧~',
  'note': {},
  'messageAccu': 0,
  'dndCommandAccu': 0,
  'commandDaily': 0,
  'messageDaily': 0,
  'dndCommandDaily': 0,
  'name': '未知群名称'},
 '264217078': {'active':

In [23]:
for gId in b.groupInfoDict:
    print(gId, b.groupInfoDict[gId]['messageAccu'], b.groupInfoDict[gId]['dndCommandAccu'],b.groupInfoDict[gId]['commandAccu'])
    print(gId, b.groupInfoDict[gId]['messageDaily'], b.groupInfoDict[gId]['dndCommandDaily'],b.groupInfoDict[gId]['commandDaily'])

922755017 0 0 0
922755017 0 0 0
861919492 73 1 4
861919492 73 1 4
588983918 0 0 0
588983918 0 0 0
264217078 4023 52 53
264217078 4023 52 53
809650245 97 1 9
809650245 97 1 9
706729478 1952 31 124
706729478 1952 31 124
818914076 303 7 11
818914076 303 7 11
713832067 492 4 6
713832067 492 4 6
753362884 848 13 38
753362884 848 13 38
779332956 2626 15 16
779332956 2626 15 16
836397763 3899 119 145
836397763 3899 119 145
1073712542 12 0 0
1073712542 12 0 0
653289152 0 0 0
653289152 0 0 0
151877541 26 4 17
151877541 26 4 17
996772855 7 0 0
996772855 7 0 0
1065251208 0 0 0
1065251208 0 0 0
647408085 1430 7 12
647408085 1430 7 12
245846401 7 7 7
245846401 7 7 7
680332562 1198 44 51
680332562 1198 44 51
885956404 452 69 76
885956404 452 69 76
558695261 170 3 14
558695261 170 3 14
69505398 382 0 0
69505398 382 0 0
717828542 293 0 0
717828542 293 0 0
1032610546 50 0 2
1032610546 50 0 2
34754947 4 0 4
34754947 4 0 4
1036494451 335 7 11
1036494451 335 7 11
723098364 1140 4 8
723098364 1140 4 8
1065

In [24]:
# import matplotlib.pyplot as plt
# credits =[]
# for uid in b.userInfoDict:
#     credits.append(b.userInfoDict[uid]['credit'])
# # print(sorted(credits, key = lambda x: -1*x))
# print(len(credits))
# plt.hist(credits)
# plt.show()

In [25]:
# %%time
# import math

# # xDyKz
# def ComputeException(x, y, z = 1):
#     return z + z*(y-1)*(2*x-z+1)/(2*x+2)+0.5
# print(ComputeException(x=4, y=6, z = 3))
# print(ComputeException(x=2, y=20, z = 1))
# n = 5000
# times = 0
# error = 0
# for x in [4, 6, 8, 10]:
#     for y in [4, 6, 8, 10]:
#         for z in [1, 2, 3, 4]:
#             for r in range(10):
#                 total = 0
#                 for i in range(n):
#                     error, output, rollResult = tool_dice.RollDiceCommand(f'{x}d{y}k{z}')
#                     total += rollResult.totalValueList[0]
#                 mean = total/n
#                 exp = ComputeException(x, y, z)
#     #             print(mean-exp, mean, exp)
#                 error += mean-exp
#                 times+=1
# print(error/times)

# # ${\sum_{i=1}^z(1+\frac{(x-i+1)(y-1)}{x+1})}$
# # ${=z+\sum_{i=1}^z\frac{(x-i+1)(y-1)}{x+1}}$
# # ${=z+\frac{y-1}{x+1}\sum_{i=1}^z(x-i+1)}$
# # ${=z+\frac{y-1}{x+1}(zx+z+\sum_{i=1}^z(-i))}$
# # ${=z+\frac{y-1}{x+1}(zx+z+\frac{-z(1+z)}{2})}$
# # ${=z+\frac{y-1}{x+1}\frac{2zx+2z-z-z^2)}{2}}$
# # ${=z+\frac{z(y-1)}{x+1}\frac{(2x-z+1)}{2}}$
# # ${=0.5+z+\frac{z(y-1)(2x-z+1)}{2x+2}}$

In [26]:
# %%time
# i = 0
# queryKeys = ['法术', '麻痹']
# result = []
# for item in b.queryInfoDict:
#     valid = True
#     for k in queryKeys:
#         if not k in b.queryInfoDict[item]:
#             valid = False
#             break
#     if valid:
#         result.append(item)
#     i += len(b.queryInfoDict[item])
# print(i)
# print(len(result), result)

In [27]:
# assert ((await b.DailyUpdate())[0].resultStr)

In [28]:
# import datetime
# datetime.timedelta(hours = 1) > datetime.timedelta(days = 30)
# datetime.datetime.now()+datetime.timedelta(days = 1) - datetime.datetime.now()

In [29]:
# for pId in b.userInfoDict.keys():
#     print(pId, b.userInfoDict[pId]['credit'])

In [30]:
# queryDict = {}

# fileName = '5E_PHB种族_by梨子'
# title = ''
# content = ''
# with open(f'../资料/{fileName}.txt', 'r', encoding="utf-8-sig") as f:
#     data = f.readline().strip()
#     while data:
#         data = data.strip()
#         if not data:
#             if title:
#                 if len(title) > 20:
#                     print("Title too long?"+title+"##"+content)
#                 elif len(content) < 20:
#                     print("Content too small?"+title+"##"+content)
#                 content = content.replace('【换行】', '\n')
#                 queryDict[title] = content[:-1]
#             title = ''
#             content = ''
#         else:
#             if not title:
#                 title = data
#             else:
#                 content += data.strip() + '\n'
#         data = f.readline()
# content = content.replace('【换行】', '\n')
# queryDict[title] = content[:-1]
# print(queryDict.keys())
# print(queryDict[list(queryDict.keys())[0]])
# utils.UpdateJson(queryDict, f'./plugins/custom_data/{fileName}_query.json')

In [31]:
# import re
# fileName = '库藏宝藏cr17+'
# deckTitle = ''
# deckList = []
# deckRelayList = []
# deckDict = {}
# totalWeight = 0
# keyList = [['WEIGHT', re.compile('WEIGHT\(.*?\)')], ['ROLL', re.compile('ROLL\(.*?\)')],
#            ['DRAW', re.compile('DRAW\(.*?\)')], ['END', re.compile('END\(\)')]]
# with open(f'../资料/牌堆/{fileName}.txt', 'r', encoding="utf-8-sig") as f:
#     data = f.readline().strip()
#     deckTitle = data
#     data = f.readline().strip()
#     if data[:3] == '依赖:':
#         deckRelayList = data[3:].split('/')
#         deckRelayList = [relay.strip() for relay in deckRelayList]
#         data = f.readline().strip()
#     while data:
#         weight = 1
#         end = False
#         rawContent = data
#         content = []
#         for key in keyList:
#             resultList = key[1].findall(data)
#             if resultList:
#                 if key[0] == 'WEIGHT':
#                     assert len(resultList) == 1, 'WEIGHT指令只能出现一次'
#                     weight = int(resultList[0][7:-1])
#                     data = data.replace(resultList[0], '')
#                     assert weight >= 1
#                 elif key[0] == 'END':
#                     assert len(resultList) == 1, 'END指令只能出现一次'
#                     end = True
#                     data = data.replace(resultList[0], '')
#                 else:
#                     for result in resultList:
#                         index = data.find(result)
#                         arg = data[index+len(key[0])+1:index+len(result)-1]
#                         if index != 0:
#                             if data[:index].split():
#                                 content.append(['TEXT',data[:index].replace('\\n', '\n')])
#                         if key[0] == 'DRAW':
#                             targetDeck = arg.split('/')[0]
#                             assert len(arg.split('/')) <= 2
#                             assert targetDeck == deckTitle or targetDeck in deckRelayList, targetDeck
#                         elif key[0] == 'ROLL':
#                             assert tool_dice.isDiceCommand(arg)
#                         content.append([key[0],arg])
#                         data = data[index+len(result):]
#         if data.split():
#             content.append(['TEXT', data.replace('\\n', '\n')])
#         assert content, rawContent
#         totalWeight += weight
#         deckList.append({'weight':weight, 'content':content, 'end':end, 'raw':rawContent})
#         data = f.readline().strip()

# deckDict['title'] = deckTitle
# deckDict['relay'] = deckRelayList
# deckDict['totalWeight'] = totalWeight
# deckDict['list'] = deckList
# utils.UpdateJson(deckDict, f'./plugins/custom_data/{fileName}_deck.json')
# print(deckDict)

In [32]:
# fileName = '混合菜谱'
# cookDict = {}
# curName = ''
# with open(f'../资料/菜谱/{fileName}.txt', 'r', encoding="utf-8-sig") as f:
#     data = f.readline().strip()
#     while data:
#         data = data.split(':')
#         if data[0] == '名称':
#             cookDict[data[1]] = {}
#             curName = data[1]
#         if data[0] == '美味' or data[0] == '难度':
#             data[1] = int(data[1])
#         if data[0] == '菜系' or data[0] == '种类' or data[0] == '风格':
#             data[1] = data[1].split('/')
#         cookDict[curName][data[0]] = data[1]
#         data = f.readline().strip()
#         if data == '':
#             data = f.readline().strip()
#             cookDict[curName]['关键词'] = cookDict[curName]['菜系'] + cookDict[curName]['种类'] + cookDict[curName]['风格']
# print(cookDict)
# utils.UpdateJson(cookDict, f'./plugins/custom_data/{fileName}_menu.json')

In [33]:
# joke_path = '../资料/笑话'
# filesPath = os.listdir(joke_path) #读取所有文件名
# filesPath = sorted(filesPath)
# jokeDict = {}
# jokeListStr = []
# jokeListImg = []
# for fp in filesPath:
#     try:
#         if fp[-4:] == '.txt':
#             print('txt', fp)
#             absPath = os.path.join(joke_path, fp)
#             with open(absPath, 'r') as f:
#                 jokeCur = ''
#                 data = f.readline()
#                 while data:
#                     jokeCur += data
#                     data = f.readline()
#                 jokeCur = jokeCur.strip()
#                 if jokeCur:
#                     jokeListStr.append(jokeCur)
#         elif fp[-4:] == '.png' or fp[-4:] == '.gif':
#             print('img', fp)
#             jokeListImg.append(fp)
#     except Exception as e:
#         print(f'error:{e}')
# assert len(jokeListStr) > 0
# print(f'笑话资料库加载成功! 共{len(jokeListStr)}个文字条目, {len(jokeListImg)}个图片条目')
# jokeDict['word'] = jokeListStr
# jokeDict['img'] = jokeListImg
# utils.UpdateJson(jokeDict, f'./plugins/custom_data/joke_info/joke.json')

In [34]:
# fileName = '职业表'
# result = ''
# indexRange = list(range(0,99)) #+[0]
# index = 0
# # split = '    '
# # split = '/'
# split = '  '
# with open(f'../资料/{fileName}.txt', 'r', encoding="utf-8-sig") as f:
#     data = f.readline().strip()
#     while data:
#         if index in indexRange:
#             result += data + split
#         index += 1
#         data = f.readline().strip()
#         if data == '':
#             data = f.readline().strip()
#             result = result[:-1].strip() + '\n'
#             index = 0
# result = result.replace('—', '-')
# print(result)

In [35]:
# synDict = {}
# fileName = 'syn_spell'
# with open(f'../资料/{fileName}.txt', 'r', encoding="utf-8-sig") as f:
#     data = f.readline().strip()
#     while data:
#         synInfo = data.split(':')
#         synInfo[0] = synInfo[0].split('/')
#         for syn in synInfo[0]:
#             synDict[syn] = synInfo[1]
#         data = f.readline().strip()
# print(synDict)
# utils.UpdateJson(synDict, f'./plugins/custom_data/{fileName}.json')